In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv
import inspect

In [2]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }


In [4]:
directory = './datasets/'
docs = []

In [5]:
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    loader = TextLoader(filepath)
    doc = loader.load()
    docs.extend(doc)

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

Created a chunk of size 1162, which is longer than the specified 1024
Created a chunk of size 1239, which is longer than the specified 1024
Created a chunk of size 1081, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024
Created a chunk of size 3424, which is longer than the specified 1024
Created a chunk of size 3257, which is longer than the specified 1024
Created a chunk of size 1985, which is longer than the specified 1024
Created a chunk of size 2477, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024


In [16]:
print(inspect.getfile(Milvus.__init__))

/Users/vaclavpechtor/Code/home_projects/LuGPT/env/lib/python3.11/site-packages/langchain/vectorstores/milvus.py


In [18]:
embeddings = OpenAIEmbeddings()

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args=connection_args,
    collection_name='Simple_DIIN_QA'
)

In [19]:
# Create text widget for input
input_text = widgets.Text()

# Create a global variable to hold the input
query = ""

# Create function to update the query variable
def update_query(change):
    global query
    query = change.new

# Tell the text widget to call update_query() function when its value changes
input_text.observe(update_query, 'value')

# Display the text widget
display(input_text)

Text(value='')

In [28]:
# query = "Was ist SAFe?"
print(f"Query: {query}")
docs = vector_store.similarity_search(query)

print(docs)

Query: Welche Projekte gibt es in der Informatik?
[Document(page_content='Folgende ICT-Berufe können bei uns gelernt werden:\nDie Digitalisierung erfasst alle Lebens- und Arbeitsbereiche der heutigen Gesellschaft. Informations- und Kommunikationstechnologien (ICT) leisten dabei einen unabdingbaren Beitrag, heutige und künftige Herausforderungen der Wirtschaft, Umwelt und Gesellschaft zu meistern. Als ICT-Fachfrau / ICT-Fachmann bist du DER Profi, wenn es um den vielfältigen und optimalen Einsatz verschiedenster ICT-Mittel geht.\nInformatiker/-innen EFZ mit Schwerpunkt Systemtechnik arbeiten vor allem in den Bereichen Planung, Installation, Anwendungen sowie im Betrieb und Wartung von Informatiknetzwerken. Ausserdem befassen sie sich mit Hard- und Software von vernetzten Anlangen, automatisieren Informatikprozesse und überwachen Systeme und Server, damit alles reibungslos abläuft.\nDie vierjährige berufliche Grundbildung umfasst eine Allgemeinbildung, die einer beruflichen KV-Grundbildu

In [29]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
# query = "Was ist SAFe?"
openai_response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [30]:
text, source = openai_response['output_text'].split('\n')
sources = source.split(',')
source_url = next(iter(sources), ' No URL found').replace('./datasets/', 'https://').replace('SOURCES:', '').replace('.txt', '').replace('__', '/').strip().replace("'", "")


In [31]:
print(f'Query: {query}')
print('Result: '+text)
print(f'Found in: {source_url}')

Query: Welche Projekte gibt es in der Informatik?
Result:  Es gibt verschiedene Projekte in der Informatik, wie z.B. Planung, Installation, Anwendungen, Betrieb und Wartung von Informatiknetzwerken, Administration und Unterstützung der Dienststelle Informatik in den Bereichen Betriebswirtschaft und Organisation, Integration in die Sicherheits- und Systeminfrastruktur, Schutz der kantonalen Daten in der privaten und öffentlichen Cloud, Werkzeugkasten mit Digitalisierungswerkzeugen, integrierte Kommunikations- und Kollaborationslösung.
Found in: https://informatik.lu.ch/Job/Informatiklehre
